## Encrypted Facial Recognition Demo
This Jupyter notebook provides an introduction to performing facial recognition over encrypted images.

In [ ]:
# Import libraries
from venumML.venumpy import small_glwe as vp
from scipy.spatial import distance
# Dependancies: pip install tensorflow, deepface, scipy

from deepface import DeepFace

In [2]:
# Import the images to perform facial recognition on
img1_path = "hp1.png"
img2_path = "hp2.png"

# Image Embeddings

Represent images as facenet embeddings.

In [3]:
# Use the deepface lib to create image embeddings
img1_embedding = DeepFace.represent(img1_path, model_name = 'Facenet')[0]["embedding"]
img2_embedding = DeepFace.represent(img2_path, model_name = 'Facenet')[0]["embedding"]

# Encryption

Apply homomorphic encryption to facial embeddings done on client side.

Store the embeddings in the cloud.

In [4]:
# Encrypt the image embeddings
ctx = vp.SecretContext()
ctx.precision= 6
c1 = [ctx.encrypt(v) for v in img1_embedding]
c2 = [ctx.encrypt(v) for v in img2_embedding]

# Calculations

In the cloud, make computations over encrypted facial embeddings.

Euclidean Distance Metric: Subtract vectors, then perform dot product.

In [5]:
# Subtract the encrypted vectors
cipher_d = [c2_i - c1_i for c2_i, c1_i in zip(c2, c1)]

In [6]:
# Perform a dot product to get the euclidean distance metric
euclidean_squared_0 = [cipher_d_i * cipher_d_i for cipher_d_i in cipher_d]

total = ctx.encrypt(0.0)
for i in euclidean_squared_0:
    total = total + i
total

# Decryption


Homomorphically encrypted euclidean squared value computed in the cloud, then retrieve it from the client.

Only the client can decrypt it because they hold the private key.

In [7]:
# Decrypt the euclidean distance facial recognition metric
decrypted_es = total.decrypt()
decrypted_es

85.186523206818

# Final Results in plaintext

In [8]:
if decrypted_es < 100:
    print("This is the same person in different images")
else:
    print("These are different people")

This is the same person in different images


# Validation

What if euclidean distance calculation is done over plaintext data?

The result should be the same.

In [9]:
# Euclidean squared distance
euclidean_distance = distance.euclidean(img1_embedding, img2_embedding)
plt_distance = euclidean_distance**2

print("euclidean squared - plaintext: ", plt_distance)
print("euclidean squared - homomorphic: ", decrypted_es)

# Check the difference in precision
if abs(plt_distance - decrypted_es) < 0.0001:
  print("The difference between plaintext and homomorphic is acceptable.")
else:
  print("The difference between plaintext and homomorphic is unacceptable.")

euclidean squared - plaintext:  85.1865116552079
euclidean squared - homomorphic:  85.186523206818
The difference between plaintext and homomorphic is acceptable.
